In [ ]:
def parse_json(json_path:str)->dict:
    """Parse a JSON file produced by Plastic Origin Mobile App

    Args:
        json_path (str): the path of a json file produced by mobile app

    Returns:
        dict: the json data as a dictionnary
    """
    with open(json_path) as json_file:
        json_data = json.load(json_file)
    return json_data

In [ ]:
json_path = '../datasample/mobile.json'

In [ ]:
json_data = parse_json(json_path)

In [ ]:
json_data.keys()

In [ ]:
def get_json_gps_list(json_data:dict)->list:
    """Get a list of GPS point from a json_data object

    Args:
        json_data (dict): the gps data as a json dict

    Returns:
        list: a list of GPS point
    """
    point_list = []
    for point in json_data['positions']:
        time = datetime.strptime(point['date'][:19].replace("T"," "),'%Y-%m-%d %H:%M:%S')
        point_info = {'Time': time, 'Latitude': point['lat'],
                              'Longitude': point['lng'], 'Elevation': 0}
        point_list.append(point_info)
    return point_list


In [ ]:
gps_point_list = get_json_gps_list(json_data)

In [ ]:
date0 = gps_point_list[0]['Time']
date370 = gps_point_list[-1]['Time']

In [ ]:
print(date0)
print(date370)

In [ ]:
date370-date0

In [ ]:
type(gps_point_list)

In [ ]:
len(gps_point_list)

In [ ]:
for i in range(0,10):
    print(gps_point_list[i])

In [ ]:
gps_points_filled = fill_gps(gps_points, video_duration_sup)

In [ ]:
filled_gps = gps_point_list.copy()
delta = filled_gps[i+1]['Time']-filled_gps[i]['Time']
delta = int(delta.total_seconds())

In [ ]:
delta = filled_gps[i+1]['Time']-filled_gps[i]['Time']

In [ ]:
gps_point_list[0]['Time']

In [ ]:
import gpxpy
import gpxpy.gpx
def parse_gpx(gpx_path:str)->object:
    """Parse a gpx file to extract gps information

    Arguments:
        gpx_path {str} -- the path of a gpx file

    Returns:
        gpx_data -- the gpx data as a gpxpy object
    """
    gpx_file = open(gpx_path,'r',encoding='utf-8')
    gpx_data = gpxpy.parse(gpx_file)
    return gpx_data

In [ ]:
def get_gps_point_list(gpx_data:object)->list:
    """Get a list of GPS point from a gpx_data object

    Arguments:
        gpx_data {object} -- the gpx data as a gpxpy object

    Returns:
        point_list -- a list of GPS points
    """
    point_list = []
    for track in gpx_data.tracks:
        for segment in track.segments:
            for point in segment.points:
                point_info = {'Time': point.time, 'Latitude': point.latitude,
                              'Longitude': point.longitude, 'Elevation': point.elevation}
                point_list.append(point_info)
    return point_list

In [ ]:
gps_point_list = get_json_gps_list(json_data)
print(len(gps_point_list))
date0 = gps_point_list[0]['Time']
date370  =  gps_point_list[-1]['Time']
print(date0)
print(date370)
delta = date370-date0
print(delta)

In [ ]:
32*60+18

In [ ]:
from datetime import timedelta
gps_points_filled = fill_gps(gps_point_list, 1938)

In [ ]:
len(gps_points_filled)

In [ ]:
for i in range(0,150):
    print(gps_points_filled[i+1837])

In [ ]:
for i in range(0,60):
    print(gps_points_filled[i+1000])

In [ ]:
trashes = json_data['trashes']

In [ ]:
for trash in trashes:
    print(trash)

In [ ]:
import pandas as pd
def get_df_json_manual_gps(json_data:dict)->pd.DataFrame:
    """Get GPS coordinate as a DataFrame from a manually collected trash JSON file

    Args:
        json_data (dict): the json data from the Plastic Origin Mobile app

    Returns:
        pd.DataFrame: the gps info as a DataFrame associated with manually collected trash
    """
    gps_list = []
    for trash in json_data['trashes']:
        time = datetime.strptime(trash['date'][:19].replace("T"," "),'%Y-%m-%d %H:%M:%S')
        gps_point = {'Time': time, 'Latitude': trash['lat'],'Longitude': trash['lng'], 'Elevation': 0}
        shape_gps_point = long_lat_to_shape_point(gps_point)
        geo_2154 = transform_geo(shape_gps_point)
        geo_2154_gps_point = {'Time': shape_gps_point['Time'],'the_geom':geo_2154, 'Latitude':shape_gps_point['Latitude'],'Longitude': shape_gps_point['Longitude'], 'Elevation':shape_gps_point['Elevation']}
        gps_list.append(geo_2154_gps_point)
    df_manual_gps = pd.DataFrame(gps_list)
    return df_manual_gps

In [ ]:
import os
import gpxpy
import gpxpy.gpx
import json
import subprocess
import datetime
import pandas as pd
from datetime import datetime
from datetime import timedelta
from shapely.geometry import Point
from functools import partial
import pyproj
from shapely.ops import transform
from tqdm import tqdm
from gopro2gpx.main import extract

In [ ]:
def long_lat_to_shape_point(gps_long_lat_point:dict)->dict:
    """Convert a long/lat of gps point to shape obect from shapely

    Arguments:
        gps_long_lat_point {dict} -- a gps point with long/lat coordinates

    Returns:
        gps_shape_point -- a gps point with shape representation of long/lat
    """
    gps_shape_point = {'Time': gps_long_lat_point['Time'], 'the_geom': Point(
        gps_long_lat_point['Longitude'], gps_long_lat_point['Latitude']),'Latitude':gps_long_lat_point['Latitude'],'Longitude':gps_long_lat_point['Longitude'], 'Elevation': gps_long_lat_point['Elevation']}
    return gps_shape_point



def transform_geo(gps_shape_point:dict)->str:
    """Transform a gps geometry representation from 4326 to 2154

    Arguments:
        gps_shape_point {dict} -- a gps point with long/lat represented as shape Point

    Returns:
        geo2 -- the shape point in geometry 2154
    """
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),  # source coordinate system
        pyproj.Proj(init='epsg:2154'))  # destination coordinate system

    geo1 = gps_shape_point['the_geom']
    geo2 = transform(project, geo1)
    return geo2

In [ ]:
json_gps_df = get_df_json_manual_gps(json_data)

In [ ]:
json_gps_df.head()

In [ ]:
def get_df_json_manual_trash(json_data:dict)->pd.DataFrame:
    """Get trash DataFrame from a JSON file where Trash have been manually collected


    Args:
        json_data (dict): the json data from the Plastic Origin Mobile app

    Returns:
        pd.DataFrame: the gps info as a DataFrame associated with manually collected trash
    """
    trash_list = []
    i = 0
    for trash in json_data['trashes']:
        trash_type_id = map_json_label_to_trash_id_PG(trash['name'])
        trash = {'id':i,'label':trash['name'],'trash_type_id':trash_type_id}
        trash_list.append(trash)
        i = i+1
        df_manual_trash = pd.DataFrame(trash_list)
    return df_manual_trash

In [170]:
def map_json_label_to_trash_id_PG(label:str)->str:
    """Map label of a trash to equivalent ID within PostGre server

    Arguments:
        label {str} -- the label of the trash

    Returns:
        id_PG -- the equivalent id within PG Trash table of trash label
    """
    switcher = { 
        "unknown":"1", #"autre dechet" in PG Data Model mapped to IA "others" label
        "agriculturalFoodWaste":"2",
        "bottles":"3", #"bouteille boisson" in PG Data Model mapped to IA "bottles" label
        "fragments":"1",#"industriel ou construction in PG Data Model mapped to IA "fragments" label
        "fishHunting":"2",
        "foodPackage":"2",
        "householdItems":"2",
        "unknown10":"1"
    }
    id_PG =  switcher.get(label, "2")
    return id_PG

In [171]:
for trash in json_data['trashes']:
    print(trash)

{&#39;date&#39;: &#39;2020-09-26T13:25:39.337Z&#39;, &#39;lat&#39;: 43.9476609, &#39;lng&#39;: 6.974562, &#39;name&#39;: &#39;unknown&#39;}
{&#39;date&#39;: &#39;2020-09-26T13:25:43.060Z&#39;, &#39;lat&#39;: 43.9476038, &#39;lng&#39;: 6.9748482, &#39;name&#39;: &#39;unknown&#39;}
{&#39;date&#39;: &#39;2020-09-26T13:25:50.818Z&#39;, &#39;lat&#39;: 43.9476038, &#39;lng&#39;: 6.9748482, &#39;name&#39;: &#39;unknown&#39;}
{&#39;date&#39;: &#39;2020-09-26T13:25:51.452Z&#39;, &#39;lat&#39;: 43.9476038, &#39;lng&#39;: 6.9748482, &#39;name&#39;: &#39;unknown&#39;}
{&#39;date&#39;: &#39;2020-09-26T13:26:12.067Z&#39;, &#39;lat&#39;: 43.9473623, &#39;lng&#39;: 6.9754792, &#39;name&#39;: &#39;agriculturalFoodWaste&#39;}
{&#39;date&#39;: &#39;2020-09-26T13:29:44.472Z&#39;, &#39;lat&#39;: 43.9487939, &#39;lng&#39;: 6.9801769, &#39;name&#39;: &#39;unknown&#39;}
{&#39;date&#39;: &#39;2020-09-26T13:30:36.816Z&#39;, &#39;lat&#39;: 43.9493439, &#39;lng&#39;: 6.9812671, &#39;name&#39;: &#39;unknown&#39;}


In [173]:
len(json_data['trashes'])

30

In [172]:
json_trash_df = get_df_json_manual_trash(json_data)
json_trash_df

,id,label,trash_type_id
0,0,unknown,1
1,1,unknown,1
2,2,unknown,1
3,3,unknown,1
4,4,agriculturalFoodWaste,2
5,5,unknown,1
6,6,unknown,1
7,7,foodPackage,2
8,8,unknown,1
9,9,unknown,1


In [ ]:
print(json_gps_df.info())
print(json_trash_df.info())

In [ ]:
def get_df_data(df_predictions:pd.DataFrame,df_trash_gps:pd.DataFrame)->pd.DataFrame:
    """Get Data to be inserted within PostGre DB as a Dataframe

    Arguments:
        df_predictions {pd.DataFrame} -- the AI prediction as a Dataframe
        df_trash_gps {pd.DataFrame} -- the gps coordinate of all trash detected by AI as a Dataframe

    Returns:
        df_data -- Data to be inserted within PostGre DB
    """
    df_data = pd.concat([df_predictions,df_trash_gps],axis=1)
    return df_data

In [169]:
data_df = get_df_data(json_trash_df,json_gps_df)
data_df

,id,label,trash_type_id,Time,the_geom,Latitude,Longitude,Elevation
0,0,unknown,1,2020-09-26 13:25:39,POINT (1018942.49629894 6324546.189385729),43.947661,6.974562,0
1,1,unknown,1,2020-09-26 13:25:43,POINT (1018965.762535683 6324541.008771674),43.947604,6.974848,0
2,2,unknown,1,2020-09-26 13:25:50,POINT (1018965.762535683 6324541.008771674),43.947604,6.974848,0
3,3,unknown,1,2020-09-26 13:25:51,POINT (1018965.762535683 6324541.008771674),43.947604,6.974848,0
4,4,agriculturalFoodWaste,2,2020-09-26 13:26:12,POINT (1019017.705437028 6324516.757451487),43.947362,6.975479,0
5,5,unknown,1,2020-09-26 13:29:44,POINT (1019386.344763959 6324694.618956082),43.948794,6.980177,0
6,6,unknown,1,2020-09-26 13:30:36,POINT (1019470.672897963 6324760.065061009),43.949344,6.981267,0
7,7,foodPackage,2,2020-09-26 13:30:57,POINT (1019493.303073367 6324779.430958421),43.949508,6.981561,0
8,8,unknown,1,2020-09-26 13:32:37,POINT (1019664.192892549 6324821.556933214),43.949809,6.983713,0
9,9,unknown,1,2020-09-26 13:33:01,POINT (1019738.226362991 6324806.671141523),43.949641,6.984625,0
